In [47]:
import pandas as pd
import numpy as np
import ast
import faiss
from openai import AzureOpenAI

In [38]:
pd_relevant_articles = pd.read_json('./local_tests_data/relevant_articles_list/relevant_articles.json').drop(columns=["index"], axis=1)
pd_relevant_articles.head()

,title,url,keywords,author,article_id,source,relevance,relevance_check_model,relevance_check_version
0,How to thrive with AI agents — tips from an HP...,https://thenextweb.com/news/tips-on-ai-agents,"[AI, strategy, HP]",None,the_next_web-20250528111956213260,the_next_web,2,gpt-4o-2024-11-20,relevance-check-v0
1,Meta AI chief: ‘Inferiority complex’ is stunti...,https://thenextweb.com/news/european-tech-stru...,"[Meta, AI, European tech]",None,the_next_web-20250528111956213288,the_next_web,2,gpt-4o-2024-11-20,relevance-check-v0
2,US shoplifting ‘epidemic’ sparks demand for Fr...,https://thenextweb.com/news/us-shoplifting-epi...,"[shoplifting, AI cameras, France]",None,the_next_web-20250528111956213298,the_next_web,2,gpt-4o-2024-11-20,relevance-check-v0
3,TNW Backstage dives into the mind-bending worl...,https://thenextweb.com/news/tnw-backstage-brai...,"[brain-computer interfaces, technology, innova...",None,the_next_web-20250528111956213306,the_next_web,2,gpt-4o-2024-11-20,relevance-check-v0
4,Startup uses ancient bacteria to turn Texas CO...,https://thenextweb.com/news/startup-uses-ancie...,"[sustainability, CO2, green chemicals]",None,the_next_web-20250528111956213312,the_next_web,2,gpt-4o-2024-11-20,relevance-check-v0


In [39]:
pd_embeddings = pd.read_csv('./local_tests_data/relevant_articles_summaries_embeddings/relevant_articles_summaries_embeddings.csv')
pd_embeddings["summary_embedding"] = pd_embeddings["summary_embedding"].apply(ast.literal_eval)

In [40]:
pd_all_info = pd_embeddings.merge(pd_relevant_articles, on="article_id", how="inner")
embedding_matrix = np.array(pd_all_info["summary_embedding"].tolist()).astype("float32")
embedding_dim = len(embedding_matrix[0, :])

In [41]:
# Create an index (flat, exact search)
index = faiss.IndexFlatL2(embedding_dim)
index.add(embedding_matrix)  # store embeddings in index

In [42]:
centroid = np.mean(embedding_matrix, axis=0).reshape(1, -1).astype("float32")
distances, indices = index.search(centroid, k=5)

In [45]:
pd_all_info.iloc[indices[0]]["title"].values

array(['OpenAI’s next big bet won’t be a wearable: report',
       'OpenAI confiesa que "muchos" inversores han salido corriendo por la estructura de la startup',
       'Las 9 startups españolas de robótica más punteras',
       'Meta AI chief: ‘Inferiority complex’ is stunting European tech',
       'Klarna used an AI avatar of its CEO to deliver earnings, it said'],
      dtype=object)

In [55]:
query = "Insights related to the food industry"

response = client.embeddings.create(
        input=[query],
        model="text-embedding-3-small",
    )

query_embedding = response.data[0].embedding
query_vector = np.array([query_embedding], dtype="float32")
distances, indices = index.search(query_vector, k=5)

In [56]:
pd_all_info.iloc[indices[0]]["title"].values

array(['US shoplifting ‘epidemic’ sparks demand for French AI cameras',
       'Los retos de la Transformación Digital',
       'TNW Backstage dives into the mind-bending world of brain-computer interfaces',
       'Israel otorga a Intel una nueva subvención por valor de 3.200 millones de dólares',
       'Startup uses ancient bacteria to turn Texas CO2 into green chemicals'],
      dtype=object)